In [ ]:
import os
import sys
import pandas as pd
import xarray as xr
import numpy as np
sys.path.append(os.path.join(os.getcwd(), ".."))
from models.components.features_selection import jmi,discretize_data,backward_model_selection_MI_JMI,model_selection_L2reg
from shared_utils.utils_data import extract_index_label, feature_checker

In [ ]:
metrics = "/workspaces/ecg_evaluation/results/quality_metrics.nc"
d = xr.load_dataset(metrics)
list_features = [
        "Corr_interlead",
        "Corr_intralead",
        "wPMF",
        "SNRECG",
        "HR",
        "Flatline",
        "TSD",
    ]
df_X_mean, df_y = extract_index_label(
        d, list_features, aggregation_method="mean"
    )
print(df_X_mean)

In [ ]:
test = "/workspaces/ecg_evaluation/results"
rep = backward_model_selection_MI_JMI(df_X_mean,df_y,test)
print(rep)

In [ ]:
rep = model_selection_L2reg(df_X_mean,df_y)
print(rep)